In [1]:
# Importing necessary modules for general data science
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn, sys, socket, requests
import altair as alt
import biom, qiime2
from qiime2 import Metadata, Artifact
from qiime2.plugins.demux.methods import emp_single
from qiime2.plugins.feature_table.visualizers import summarize as featsum
from qiime2.plugins.demux.visualizers import summarize
from qiime2.plugins.metadata.visualizers import tabulate

In [2]:
# Loads in the qiime2 artifact
single_end_seqs_dat = qiime2.Artifact.load("zipfiles/emp-single-end-sequences.qza")
sammy_metdat = qiime2.Metadata.load('sample-metadata.tsv')
html_metadata, = tabulate(sammy_metdat)
html_metadata


<visualization: Visualization uuid: 33761492-6c44-45fa-81c7-cc76afaedbc2>

In [3]:
barcode_seq_dat = sammy_metdat.get_column('barcode-sequence')
emp_single_alph = emp_single(single_end_seqs_dat, barcodes=barcode_seq_dat)

In [4]:
summarize(emp_single_alph.per_sample_sequences).visualization

<visualization: Visualization uuid: 435367b2-8c90-4177-968a-b77effd6a504>

<Figure size 432x288 with 0 Axes>

In [5]:
from qiime2.plugins.dada2.methods import denoise_single
denoised_demux_seq_dat = denoise_single(emp_single_alph.per_sample_sequences, trunc_len=92)
denoised_demux_seq_dat

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/qiime2-archive-94d2_84t/5193cde8-42e7-4b82-a335-87facc6638a8/data /tmp/tmpki9su_x8/output.tsv.biom /tmp/tmpki9su_x8/track.tsv /tmp/tmpki9su_x8 92 0 2.0 2 Inf independent consensus 1.0 1 1000000 NULL 16



Results (name = value)
---------------------------------------------------------------------------------------------------------
table                    = <artifact: FeatureTable[Frequency] uuid: 9b52f5a2-6992-4ef2-93e3-bfc2dd63614f>
representative_sequences = <artifact: FeatureData[Sequence] uuid: fb63b7d4-6dda-4e7f-8cf3-5ad81b91645d>
denoising_stats          = <artifact: SampleData[DADA2Stats] uuid: 290c1e55-1796-41b3-a7d7-2778e4bcbc71>

In [12]:
raretable = denoised_demux_seq_dat.table.view(pd.DataFrame)
raretable.to_json


<bound method NDFrame.to_json of         bd55f5dd58cea659ed9865b606e656ef  583b244d077143a04654f71102c7090f  \
L1S105                            2598.0                               6.0   
L1S140                            2777.0                               0.0   
L1S208                            2760.0                               0.0   
L1S257                            1504.0                               0.0   
L1S281                            2100.0                               0.0   
L1S57                             3312.0                               0.0   
L1S76                             3809.0                               0.0   
L1S8                              3055.0                               0.0   
L2S155                              17.0                             580.0   
L2S175                              14.0                            1245.0   
L2S204                              25.0                             147.0   
L2S222                         

In [7]:
denoised_demux_seq_dat.representative_sequences.view(pd.Series)

bd55f5dd58cea659ed9865b606e656ef    (((T)), ((A)), ((C)), ((G)), ((G)), ((A)), ((G...
583b244d077143a04654f71102c7090f    (((T)), ((A)), ((C)), ((G)), ((T)), ((A)), ((G...
099e40cc8ba9a255e759f82ca1e1c528    (((T)), ((A)), ((C)), ((G)), ((T)), ((A)), ((G...
535194b1a1c1f26b2648baf5cdc5ed0c    (((T)), ((A)), ((C)), ((G)), ((T)), ((A)), ((G...
79871e23b65f2613568710b993c19d5c    (((T)), ((A)), ((C)), ((G)), ((G)), ((A)), ((G...
                                                          ...                        
224409cd8187fea1923668117407b381    (((T)), ((A)), ((C)), ((G)), ((T)), ((A)), ((G...
1313be370f11df6f07793bb2468cd595    (((T)), ((A)), ((C)), ((G)), ((T)), ((A)), ((G...
9f254600a6e240888423c742803c5f9e    (((T)), ((A)), ((C)), ((G)), ((T)), ((A)), ((G...
d2041b29864713a7bd9e80d9b05a38f8    (((G)), ((A)), ((C)), ((G)), ((G)), ((A)), ((G...
253d6cbf1078c1a6045f6180e3989d95    (((T)), ((A)), ((C)), ((G)), ((G)), ((G)), ((G...
Length: 823, dtype: object

In [8]:
feat_vis_sum = featsum(denoised_demux_seq_dat.table, sample_metadata=sammy_metdat)
feat_vis_sum.visualization

<visualization: Visualization uuid: ceee5cad-75a1-4c82-9a8f-a0765eb5a262>

In [9]:
from qiime2.plugins.feature_table.methods import rarefy
rare_table_dat = rarefy(table=denoised_demux_seq_dat.table, sampling_depth=1600)
featsum(rare_table_dat.rarefied_table).visualization

<visualization: Visualization uuid: c76605d8-07d0-4e9a-8fb9-8d009f32330d>

In [10]:
raretable = rare_table_dat.rarefied_table
regtable = denoised_demux_seq_dat.table